In [1]:
from sklearn.metrics import mean_absolute_error as MSE
from tensorflow.keras.models import Sequential
from cmcrameri import cm
from tqdm import tqdm
import seaborn as sns
from common import *
from models import *

2021-12-17 17:21:57.553791: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-17 17:21:57.553819: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Loading our data

In [2]:
df = pd.read_csv("../data/input_data/MAIN_DATASET.csv")
price = df['NO2_price'].values.reshape(-1,1)

# Scaling our data

In [3]:
scaler = MinMaxScaler()
price = scaler.fit_transform(price)

# Structuring our data, sliding window

In [4]:
lookbehind = 7
input_width = lookbehind*24
horizon = 24
no_hours = input_width + horizon
stride = 24

price_dataset = []


for i in range(0, len(price)-input_width, horizon):
    price_dataset.append(price[i:i+no_hours])

price_dataset = np.array(price_dataset)

n,m,k = price_dataset.shape

# Train test split

In [5]:
train = int(0.7*n)
valid = int(0.9*n)
X_train = price_dataset[:train, :input_width]
X_valid = price_dataset[train:valid, :input_width]
X_test = price_dataset[valid:, :input_width]

Y = np.empty((n, input_width, horizon))
for step_ahead in range(1, horizon + 1):
    Y[:,:, step_ahead - 1] = price_dataset[:,step_ahead:step_ahead + input_width, 0]

Y_train = Y[:train]
Y_valid = Y[train:valid]
Y_test = Y[valid:]

## Hyperparameters

In [6]:
# Common
epochs = 25
batch_size = [100, 200, 300]
eta_list = np.logspace(-2, -4, 3)
neurons_list = [128, 64, 32]
filters = 64
kernel_size = 3
strides = 3

learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                            patience=3, 
                                                            verbose=1, 
                                                            factor=0.5, 
                                                            min_lr=0.000001)
callbacks = [learning_rate_reduction]

# Setting up the univariate SimpleRNN

In [7]:
class RNN(Sequential):
    def __init__(self, eta, neurons, horizon=24):
        super(self.__class__, self).__init__()

        # RNN architecture
        self.add(keras.layers.SimpleRNN(neurons, return_sequences=True, input_shape=[None, k]))
        self.add(keras.layers.SimpleRNN(neurons, return_sequences=True))
        self.add(keras.layers.Dense(horizon))

        # Model compile settings
        opt = keras.optimizers.Adam(learning_rate=eta)

        # Compile model
        self.compile(loss='mse', optimizer=opt)


# return_sequences=True --> (256, 168, 24)
# return_sequences=False --> (256, 24)

## Grid search

In [8]:
results_columns = ["model","batch_size", "learning_rate", "epoch", "neurons", "mse_all"]
results_columns += [f"mse{i}" for i in range(1,horizon+1)] 
results = pd.DataFrame(columns=results_columns)

x_columns = [f"x{k}" for k in range(1, input_width+1)]
t_columns = [f"t{k}" for k in range(1, horizon+1)]
t_hat_columns = [f"t_hat{k}" for k in range(1, horizon+1)]
preds_columns = columns=x_columns+t_columns+t_hat_columns
results_preds = pd.DataFrame(columns=preds_columns)

epochs_colmns = [f"epoch{i}" for i in range(epochs)]
Y_test_last = Y_test[:,-1]
best_mse = np.inf

i = 0
for eta in tqdm(eta_list):
    for batch in batch_size:
        for neurons in neurons_list:
            RNN_model = RNN(eta, neurons, horizon=horizon)

            # Training
            history = RNN_model.fit(X_train, Y_train, 
                        epochs=epochs, batch_size = batch, 
                        shuffle=False, callbacks= callbacks,  
                        validation_data=(X_valid, Y_valid), verbose=0)

            # Prediction
            Y_pred_rnn = RNN_model.predict(X_test)
            Y_pred_rnn_last = Y_pred_rnn[:,-1]     
            
            # Results
            mse_val = MSE(Y_test_last, Y_pred_rnn_last)
            mse_all = np.mean((Y_test_last - Y_pred_rnn_last) ** 2, axis=0) # The mean of each column 
            
            
            # Storing results for all models
            res1 = np.array([i, batch, eta, epochs, neurons, mse_val])
            res2 = np.array(mse_all)
            res = np.concatenate([res1,res2], axis=0)
            results.loc[i] = res
            results.to_csv(f"{REPORT_DATA}univariate_simpleRNN_results.csv")

            # Storing results from the best model
            if best_mse > mse_val:
                best_mse = mse_val
                print(f"\nNew best model with MSE = {mse_val} !\n")
                # Predictions and target for best results
                preds_res =  np.concatenate([X_test[...,0], Y_test_last, Y_pred_rnn_last],axis=1)
                best_results_preds = pd.DataFrame(preds_res, columns=preds_columns)
                best_results_preds.to_csv(f"{REPORT_DATA}best_models/univariate_simpleRNN_preds_best.csv")

                
                # Best results relative to parameters
                best_results = pd.DataFrame(columns=results_columns)
                best_results.loc[i] = res
                best_results.to_csv(f"{REPORT_DATA}best_models/univariate_simpleRNN_results_best.csv")
                
                # Model evaluation result            
                best_train_val_losses = pd.DataFrame(columns=["training_loss", "validation_loss"])
                best_train_val_losses["training_loss"] = history.history["loss"]
                best_train_val_losses["validation_loss"] = history.history["val_loss"]
                best_train_val_losses.to_csv(f"{REPORT_DATA}best_models/univariate_simpleRNN_train_val_losses_best.csv")            
                
            i += 1

  0%|          | 0/3 [00:00<?, ?it/s]2021-12-17 16:44:58.309112: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-12-17 16:44:58.309451: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-17 16:44:58.309509: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-12-17 16:44:58.309552: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-12-17 16:44:58.309637: W tensorflow/stream_exec


Epoch 00009: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00012: ReduceLROnPlateau reducing learning rate to 0.0025.

Epoch 00021: ReduceLROnPlateau reducing learning rate to 0.00125.

Epoch 00024: ReduceLROnPlateau reducing learning rate to 0.000625.

New best model with MSE = 0.09223925043952012 !


Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00012: ReduceLROnPlateau reducing learning rate to 0.0025.

Epoch 00015: ReduceLROnPlateau reducing learning rate to 0.00125.

Epoch 00019: ReduceLROnPlateau reducing learning rate to 0.000625.

Epoch 00022: ReduceLROnPlateau reducing learning rate to 0.0003125.

Epoch 00025: ReduceLROnPlateau reducing learning rate to 0.00015625.

New best model with MSE = 0.029450124735978054 !


Epoch 00011: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00016: ReduceLROnPlateau reducing learning rate to 0.0025.

Epoch 00019: ReduceLROnPlateau reducing learning rate to 0.00125.

Epoch 00022: ReduceLROnPlatea

 33%|███▎      | 1/3 [04:39<09:19, 279.76s/it]


Epoch 00016: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00019: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00022: ReduceLROnPlateau reducing learning rate to 0.000125.

Epoch 00025: ReduceLROnPlateau reducing learning rate to 6.25e-05.

Epoch 00013: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00016: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00019: ReduceLROnPlateau reducing learning rate to 0.000125.

Epoch 00022: ReduceLROnPlateau reducing learning rate to 6.25e-05.

Epoch 00025: ReduceLROnPlateau reducing learning rate to 3.125e-05.

Epoch 00018: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00022: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00025: ReduceLROnPlateau reducing learning rate to 0.000125.

Epoch 00013: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00018: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00021: ReduceLROnPlateau reducing learning rate to 0

 67%|██████▋   | 2/3 [09:40<04:52, 292.37s/it]


Epoch 00017: ReduceLROnPlateau reducing learning rate to 5e-05.

Epoch 00024: ReduceLROnPlateau reducing learning rate to 2.5e-05.


100%|██████████| 3/3 [14:28<00:00, 289.38s/it]


# Setting up the univariate ConvGRU

In [9]:
class ConvGRU(Sequential):
    def __init__(self, eta, neurons_first, neurons_second, horizon=24):
        super(self.__class__, self).__init__()

        # ConvGRU architecture
        self.add(keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=strides, padding="valid", input_shape=[None, k]))
        self.add(keras.layers.GRU(neurons_first, return_sequences=True))
        self.add(keras.layers.GRU(neurons_second, return_sequences=True))
        self.add(keras.layers.Dense(horizon))

        # Model compile settings
        opt = keras.optimizers.Adam(learning_rate=eta)

        # Compile model
        self.compile(loss='mse', optimizer=opt)

## Grid search

In [10]:
results_columns = ["model","batch_size", "learning_rate", "epoch", "neurons", "mse_all"]
results_columns += [f"mse{i}" for i in range(1,horizon+1)] 
results = pd.DataFrame(columns=results_columns)

x_columns = [f"x{k}" for k in range(1, input_width+1)]
t_columns = [f"t{k}" for k in range(1, horizon+1)]
t_hat_columns = [f"t_hat{k}" for k in range(1, horizon+1)]
preds_columns = columns=x_columns+t_columns+t_hat_columns
results_preds = pd.DataFrame(columns=preds_columns)

epochs_colmns = [f"epoch{i}" for i in range(epochs)]
Y_test_last = Y_test[:,-1]
best_mse = np.inf

i = 0
for eta in tqdm(eta_list):
    for batch in batch_size:
        for neurons in neurons_list:
            ConvGRU_model = ConvGRU(eta, neurons, neurons, horizon=horizon)

            # Training
            history = ConvGRU_model.fit(X_train, Y_train[:,2::strides], 
                        epochs=epochs, batch_size = batch, 
                        shuffle=False, callbacks= callbacks,  
                        validation_data=(X_valid, Y_valid[:,2::strides]), verbose=0)

            # Prediction
            Y_pred_rnn = ConvGRU_model.predict(X_test)
            Y_pred_rnn_last = Y_pred_rnn[:,-1]     
            
            # Results
            mse_val = MSE(Y_test_last, Y_pred_rnn_last)
            mse_all = np.mean((Y_test_last - Y_pred_rnn_last) ** 2, axis=0) # The mean of each column 
                        
            # Storing results for all models
            res1 = np.array([i, batch, eta, epochs, neurons, mse_val])
            res2 = np.array(mse_all)
            res = np.concatenate([res1,res2], axis=0)
            results.loc[i] = res
            results.to_csv(f"{REPORT_DATA}univariate_convGRU_results.csv")

            # Storing results from the best model
            if best_mse > mse_val:
                best_mse = mse_val
                print(f"\nNew best model with MSE = {mse_val} !\n")
                # Predictions and target for best results
                preds_res =  np.concatenate([X_test[...,0], Y_test_last, Y_pred_rnn_last],axis=1)
                best_results_preds = pd.DataFrame(preds_res, columns=preds_columns)
                best_results_preds.to_csv(f"{REPORT_DATA}best_models/univariate_convGRU_preds_best.csv")

                
                # Best results relative to parameters
                best_results = pd.DataFrame(columns=results_columns)
                best_results.loc[i] = res
                best_results.to_csv(f"{REPORT_DATA}best_models/univariate_convGRU_results_best.csv")
                
                # Model evaluation result            
                best_train_val_losses = pd.DataFrame(columns=["training_loss", "validation_loss"])
                best_train_val_losses["training_loss"] = history.history["loss"]
                best_train_val_losses["validation_loss"] = history.history["val_loss"]
                best_train_val_losses.to_csv(f"{REPORT_DATA}best_models/univariate_convGRU_train_val_losses_best.csv")            
                
            i += 1

  0%|          | 0/3 [00:00<?, ?it/s]


Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.005.

New best model with MSE = 0.04304351121387934 !


Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.0025.

Epoch 00016: ReduceLROnPlateau reducing learning rate to 0.00125.

Epoch 00019: ReduceLROnPlateau reducing learning rate to 0.000625.

Epoch 00023: ReduceLROnPlateau reducing learning rate to 0.0003125.

New best model with MSE = 0.02848292118488004 !


Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.0025.

Epoch 00014: ReduceLROnPlateau reducing learning rate to 0.00125.

Epoch 00018: ReduceLROnPlateau reducing learning rate to 0.000625.

Epoch 00021: ReduceLROnPlateau reducing learning rate to 0.0003125.

Epoch 00024: ReduceLROnPlateau reducing learning rate to 0.00015625.

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00012: ReduceLROnPlatea

 33%|███▎      | 1/3 [05:01<10:02, 301.44s/it]


New best model with MSE = 0.028368002043408447 !


Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00013: ReduceLROnPlateau reducing learning rate to 0.000125.

Epoch 00017: ReduceLROnPlateau reducing learning rate to 6.25e-05.

Epoch 00020: ReduceLROnPlateau reducing learning rate to 3.125e-05.

Epoch 00023: ReduceLROnPlateau reducing learning rate to 1.5625e-05.

Epoch 00012: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00015: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00018: ReduceLROnPlateau reducing learning rate to 0.000125.

Epoch 00021: ReduceLROnPlateau reducing learning rate to 6.25e-05.

Epoch 00024: ReduceLROnPlateau reducing learning rate to 3.125e-05.

Epoch 00014: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00017: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00021: ReduceLROnPlateau reducing learning rate to 0.000125.

E

 67%|██████▋   | 2/3 [09:16<04:34, 274.03s/it]


Epoch 00007: ReduceLROnPlateau reducing learning rate to 5e-05.

Epoch 00008: ReduceLROnPlateau reducing learning rate to 5e-05.

Epoch 00013: ReduceLROnPlateau reducing learning rate to 2.5e-05.

Epoch 00017: ReduceLROnPlateau reducing learning rate to 1.25e-05.

Epoch 00020: ReduceLROnPlateau reducing learning rate to 6.25e-06.

Epoch 00023: ReduceLROnPlateau reducing learning rate to 3.125e-06.

Epoch 00008: ReduceLROnPlateau reducing learning rate to 5e-05.

Epoch 00016: ReduceLROnPlateau reducing learning rate to 2.5e-05.

Epoch 00021: ReduceLROnPlateau reducing learning rate to 1.25e-05.

Epoch 00024: ReduceLROnPlateau reducing learning rate to 6.25e-06.

Epoch 00011: ReduceLROnPlateau reducing learning rate to 5e-05.

Epoch 00014: ReduceLROnPlateau reducing learning rate to 2.5e-05.

Epoch 00017: ReduceLROnPlateau reducing learning rate to 1.25e-05.

Epoch 00020: ReduceLROnPlateau reducing learning rate to 6.25e-06.

Epoch 00023: ReduceLROnPlateau reducing learning rate to 3.12

100%|██████████| 3/3 [14:09<00:00, 283.24s/it]


# Setting up the univariate LSTM

In [7]:
class LSTM(Sequential):
    def __init__(self, eta, neurons, horizon=24):
        super(self.__class__, self).__init__()
        
        # self.add(keras.layers.Bidirectional(keras.layers.LSTM(neurons, return_sequences=True)))
        # self.add(keras.layers.Bidirectional(keras.layers.LSTM(neurons, return_sequences=True)))
        
        self.add(keras.layers.LSTM(neurons, return_sequences=True, input_shape=[None, k]))
        self.add(keras.layers.LSTM(neurons, return_sequences=True))
        self.add(keras.layers.Dense(horizon))

        # Model compile settings
        opt = keras.optimizers.Adam(learning_rate=eta)

        # Compile model
        self.compile(loss='mse', optimizer=opt)

## Grid search

In [8]:
results_columns = ["model","batch_size", "learning_rate", "epoch", "neurons", "mse_all"]
results_columns += [f"mse{i}" for i in range(1,horizon+1)] 
results = pd.DataFrame(columns=results_columns)

x_columns = [f"x{k}" for k in range(1, input_width+1)]
t_columns = [f"t{k}" for k in range(1, horizon+1)]
t_hat_columns = [f"t_hat{k}" for k in range(1, horizon+1)]
preds_columns = columns=x_columns+t_columns+t_hat_columns
results_preds = pd.DataFrame(columns=preds_columns)

epochs_colmns = [f"epoch{i}" for i in range(epochs)]

Y_test_last = Y_test[:,-1]

best_mse = np.inf

i = 0
for eta in tqdm(eta_list):
    for batch in batch_size:
        for neurons in neurons_list:
            LSTM_model = LSTM(eta, neurons, horizon=horizon)

            # Training
            history = LSTM_model.fit(X_train, Y_train, 
                        epochs=epochs, batch_size = batch, 
                        shuffle=False, callbacks= callbacks,  
                        validation_data=(X_valid, Y_valid), verbose=0)

            # Prediction
            Y_pred_rnn = LSTM_model.predict(X_test)
            Y_pred_rnn_last = Y_pred_rnn[:,-1]     
            
            # Results
            mse_val = MSE(Y_test_last, Y_pred_rnn_last)
            mse_all = np.mean((Y_test_last - Y_pred_rnn_last) ** 2, axis=0) # The mean of each column 
            
            
            # Storing results for all models
            res1 = np.array([i, batch, eta, epochs, neurons, mse_val])
            res2 = np.array(mse_all)
            res = np.concatenate([res1,res2], axis=0)
            results.loc[i] = res
            results.to_csv(f"{REPORT_DATA}univariate_LSTM_results.csv")

            # Storing results from the best model
            if best_mse > mse_val:
                best_mse = mse_val
                print(f"\nNew best model with MSE = {mse_val} !\n")
                # Predictions and target for best results
                preds_res =  np.concatenate([X_test[...,0], Y_test_last, Y_pred_rnn_last],axis=1)
                best_results_preds = pd.DataFrame(preds_res, columns=preds_columns)
                best_results_preds.to_csv(f"{REPORT_DATA}best_models/univariate_LSTM_preds_best.csv")

                
                # Best results relative to parameters
                best_results = pd.DataFrame(columns=results_columns)
                best_results.loc[i] = res
                best_results.to_csv(f"{REPORT_DATA}best_models/univariate_LSTM_results_best.csv")
                
                # Model evaluation result            
                best_train_val_losses = pd.DataFrame(columns=["training_loss", "validation_loss"])
                best_train_val_losses["training_loss"] = history.history["loss"]
                best_train_val_losses["validation_loss"] = history.history["val_loss"]
                best_train_val_losses.to_csv(f"{REPORT_DATA}best_models/univariate_LSTM_train_val_losses_best.csv")            
                
            i += 1

  0%|          | 0/3 [00:00<?, ?it/s]2021-12-17 17:22:21.514895: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-12-17 17:22:21.515081: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-17 17:22:21.515896: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-12-17 17:22:21.516282: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-12-17 17:22:21.516504: W tensorflow/stream_exec


Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.0025.

Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.00125.

Epoch 00013: ReduceLROnPlateau reducing learning rate to 0.000625.

Epoch 00022: ReduceLROnPlateau reducing learning rate to 0.0003125.

Epoch 00025: ReduceLROnPlateau reducing learning rate to 0.00015625.

New best model with MSE = 0.030514796577305756 !


Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.0025.

Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.00125.

Epoch 00013: ReduceLROnPlateau reducing learning rate to 0.000625.

Epoch 00023: ReduceLROnPlateau reducing learning rate to 0.0003125.

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.0025.

Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.00125.

Epoch 0001

 33%|███▎      | 1/3 [13:56<27:52, 836.06s/it]


Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00013: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00016: ReduceLROnPlateau reducing learning rate to 0.000125.

Epoch 00019: ReduceLROnPlateau reducing learning rate to 6.25e-05.

Epoch 00022: ReduceLROnPlateau reducing learning rate to 3.125e-05.

Epoch 00025: ReduceLROnPlateau reducing learning rate to 1.5625e-05.

New best model with MSE = 0.030250425207724394 !


Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00012: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00015: ReduceLROnPlateau reducing learning rate to 0.000125.

Epoch 00018: ReduceLROnPlateau reducing learning rate to 6.25e-05.

Epoch 00021: ReduceLROnPlateau reducing learning rate to 3.125e-05.

Epoch 00024: ReduceLROnPlateau reducing learning rate to 1.5625e-05.

New best model with MSE = 0.03011417060768677 !


Epoch 00012: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00017: Red

 67%|██████▋   | 2/3 [30:30<15:29, 929.20s/it]


Epoch 00023: ReduceLROnPlateau reducing learning rate to 5e-05.

Epoch 00013: ReduceLROnPlateau reducing learning rate to 5e-05.

Epoch 00020: ReduceLROnPlateau reducing learning rate to 2.5e-05.


100%|██████████| 3/3 [45:40<00:00, 913.47s/it]
